In [ ]:
import time
import pandas as pd
import plotly.express as px
from sentiment3d import Sentiment3D

from utils import separate_utterances

s3d = Sentiment3D()

## Compute VAC sentiment for a string

Change txt to whatever you like to see the sentiment scores. Computing sentiment for a single string should be fast, even without GPU acceleration.

In [ ]:
txt = "Heal yourself, with beautiful love, and always remember... you are the medicine."
s3d.get_utterance_sentiment(txt)

## 3D Sentiment Plot

Next we'll try something more ambitious by computing the VAC sentiment scores for an entire therapy session and plot the resulting values in a 3d figure. Note that computing the sentiment values for every utterance in the session may take 10-20 minutes if you don't have GPU acceleration.

In [ ]:
tmp_df = pd.read_csv("./data/carl_and_gloria.csv", sep="\t", index_col=0)
utterance_df = separate_utterances(tmp_df)
utterance_df.head()

In [ ]:
start = time.time()
utterances = utterance_df["utterance"].to_list()
sentiment_dict = s3d(utterances)
print(f"Computed sentiment for {len(utterances)} utterances in {(time.time()-start)/60:0.2f} minutes.")

In [ ]:
utt_res = pd.json_normalize(sentiment_dict, max_level=2)
sent_df = utterance_df.merge(utt_res, left_index=True, right_index=True, validate="1:1")
sent_df.head()

In [ ]:
FONT_COLOR = '#666666'
FILL_COLOR = 'rgb(.7,.7,.7)'
COLORS = ['rgba(62,148,204,127)', 'rgba(213,134,58,127)', 'rgba(126,154,102,127)']

utt_av = sent_df.copy()
speakers = utt_av.speaker.unique().tolist()
utt_av['length'] = utt_av['utterance'].apply(lambda u: len(u.split()))
utt_av['weight'] = utt_av['length'] / utt_av['length'].sum()

utt_av['utterance'] = utt_av['utterance'].str.wrap(30)
utt_av['utterance'] = utt_av['utterance'].apply(lambda x: x.replace('\n', '<br>'))

fig = px.scatter_3d(utt_av, x="valence", y="arousal", z="confidence", 
                    opacity=0.5, color_discrete_sequence=COLORS,
                    color="speaker", 
                    size="length", 
                    hover_data=["utterance"],
                   )

ht = "Therapist<br>VAC: %{x:.2f}, %{y:.2f}, %{z:.2f}<br>%{customdata[0]}<extra></extra>"
ax = dict(nticks=5, range=[-1.0, 1.0], spikecolor="rgba(.0,.0,.0,.3)", spikethickness=2)
fig.update_layout(plot_bgcolor='rgba(0,0,0,0)',
                  width=600, height=600,
                  margin=dict(r=0, l=0, b=0, t=0),
                  scene_aspectmode='cube',
                  scene = dict(xaxis=ax, yaxis=ax, zaxis=ax),
                  #hovermode='x unified',
                  hoverlabel=dict(
                      bgcolor="rgba(.95,.95,.95,.1)",
                      bordercolor="rgba(.0,.0,.0,.2)",
                      font={"color": "rgba(.0,.0,.0,.5)", "size": 12},
                      ),
                  legend=dict(yanchor='top', y=0.9, xanchor='right', x=1, 
                                  bgcolor='rgba(1,1,1,0)', title='', font=dict(size=14)),
                  )
fig.update_traces(hovertemplate=ht)
# To save a local interactive version of the plot uncomment the following (note: this will be a large file!):
#fig.write_html("./cg_sentiment3d.html")
fig